In [23]:
import cv2
import numpy as np
import imutils
from transform import four_point_transform
from skimage.filters import threshold_local


image = cv2.imread('images/bill.png')
orig= image.copy()
ratio = image.shape[0]/500.0
image = imutils.resize(image, height=500)
blurred = cv2.pyrMeanShiftFiltering(image, 10,20)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2.imshow("blurred image", blurred)


ret, threshold = cv2.threshold(gray, 0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
(_, cnts, _) = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

print ("Contours in the image, %d" % (len(cnts)))

shape = image.copy()
cv2.drawContours(shape, cnts, -1, (0, 255, 0), 2)
cv2.imshow("All contours", shape)


shape2  = image.copy()

# now i want to get the biggest contour
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]
cv2.drawContours(shape2, cnts, 0, (0,0,255),2)
cv2.imshow("biggest contour",shape2)
# abc = cnts[0]


for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if len(approx) == 4:
        print("Got the box....inside the if ")
        screenCnt = approx
        break

shape3 = image.copy()
cv2.drawContours(shape3, [screenCnt], -1, (0, 0, 255), 2)
cv2.imshow("Box", shape3)



warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
 
# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255
 

# show all the images
# original and final reshaped image can be very large so fix some height

cv2.imshow("Original image", imutils.resize(orig, height = 650) )
cv2.imshow("Final Scanned", imutils.resize(warped, height = 650) )

k = cv2.waitKey(0)
if k == 27 or k == ord('q'):
    cv2.destroyAllWindows()

if cv2.waitKey(0) == ord('q'):
    
# cv2.waitKey(0)
    cv2.destroyAllWindows()


Contours in the image, 274
Got the box....inside the if 
